In [1]:
import pandas as pd
import csv
import requests
import update_2021
import datetime as dt
import calendar




In [ ]:
### runs the py file that updates the csv
%run update_2021

In [2]:
### read csv

year = '2021'
df=pd.read_csv(f'Police_Incidents_{year}.csv',encoding="utf-8")
df.head()
df.size
# print(type(df))



74382

In [3]:
df.head()

,X,Y,publicaddress,caseNumber,precinct,reportedDate,reportedTime,beginDate,reportedDateTime,beginTime,...,enteredDate,centergbsid,centerLong,centerLat,centerX,centerY,neighborhood,lastchanged,LastUpdateDateETL,OBJECTID
0,-93.280188,44.985042,0006XX 5TH ST N,MP20212194,1,2021/01/04 00:00:00+00,741,2021/01/04 00:00:00+00,2021/01/04 07:41:00+00,741,...,2021/01/05 00:00:00+00,16909.0,-93.280193,44.985047,-10383903,5619167,North Loop,2021/01/05 00:00:00+00,2021/01/06 07:00:33+00,1
1,-93.279325,44.967039,0015XX LASALLE AVE,MP20212207,1,2021/01/04 00:00:00+00,900,2021/01/03 00:00:00+00,2021/01/04 09:00:00+00,1730,...,2021/01/05 00:00:00+00,15055.0,-93.279330,44.967045,-10383807,5616334,Loring Park,2021/01/05 00:00:00+00,2021/01/06 07:00:33+00,2
2,-93.296851,45.019469,0035XX GIRARD AVE N,MP20212245,4,2021/01/05 00:00:00+00,900,2020/11/03 00:00:00+00,2021/01/05 09:00:00+00,1800,...,2021/01/05 00:00:00+00,19932.0,-93.296852,45.019471,-10385758,5624587,Folwell,2021/01/05 00:00:00+00,2021/01/06 07:00:33+00,3
3,-93.305826,44.982107,0004XX NEWTON AVE N,MP20212252,4,2021/01/04 00:00:00+00,1000,2021/01/04 00:00:00+00,2021/01/04 10:00:00+00,930,...,2021/01/05 00:00:00+00,22100.0,-93.305827,44.982108,-10386757,5618705,Harrison,2021/01/05 00:00:00+00,2021/01/06 07:00:33+00,4
4,-93.268968,44.944078,0032XX 5TH AVE S,MP20212263,3,2021/01/04 00:00:00+00,951,2021/01/04 00:00:00+00,2021/01/04 09:51:00+00,945,...,2021/01/05 00:00:00+00,20656.0,-93.268977,44.944080,-10382654,5612722,Central,2021/01/05 00:00:00+00,2021/01/06 07:00:33+00,5


In [4]:
# df.groupby('offense').count()["X"]

df.count()
df.columns


Index(['X', 'Y', 'publicaddress', 'caseNumber', 'precinct', 'reportedDate',
       'reportedTime', 'beginDate', 'reportedDateTime', 'beginTime', 'offense',
       'description', 'UCRCode', 'enteredDate', 'centergbsid', 'centerLong',
       'centerLat', 'centerX', 'centerY', 'neighborhood', 'lastchanged',
       'LastUpdateDateETL', 'OBJECTID'],
      dtype='object')

In [5]:
#### remove unwanted columns

df =df[['reportedDateTime','offense',
       'description','centerLong',
       'centerLat','neighborhood']]

### remove unnecessary spaces from description
df['description']=df['description'].str.strip()


### remove unnecessary sapces from offense
df['offense']=df['offense'].str.strip()

### extracting time from reporedDateTime
df['time']= [x.time() for x in pd.to_datetime(df['reportedDateTime'])]

### extracting date from reporedDateTime

df['date']= [x.date() for x in pd.to_datetime(df['reportedDateTime'])]

### dropping reportedDateTime since we don't need it
df = df.drop(columns=['reportedDateTime'])



df.head()


,offense,description,centerLong,centerLat,neighborhood,time,date
0,AUTOTH,AUTOMOBILE THEFT,-93.280193,44.985047,North Loop,07:41:00,2021-01-04
1,AUTOTH,AUTOMOBILE THEFT,-93.279330,44.967045,Loring Park,09:00:00,2021-01-04
2,THEFT,OTHER THEFT,-93.296852,45.019471,Folwell,09:00:00,2021-01-05
3,AUTOTH,AUTOMOBILE THEFT,-93.305827,44.982108,Harrison,10:00:00,2021-01-04
4,CARJCK,ROBBERY INCLUDING AUTO THEFT,-93.268977,44.944080,Central,09:51:00,2021-01-04


In [6]:

# #### read in severity csv into a dataframe
sev_df=pd.read_csv('crime_severity.csv',encoding="utf-8")
#### replace column name crime with description
sev_df=sev_df.rename(columns={'crime':'description'})


#### remove spacing from description
sev_df['description']=sev_df['description'].str.strip()


#### merge the severity and main dataframe, adding the severity to each crime in the main dataframe
df = pd.merge(df,sev_df,on='description',how='outer')

df.head(10)



,offense,description,centerLong,centerLat,neighborhood,time,date,severity
0,AUTOTH,AUTOMOBILE THEFT,-93.280193,44.985047,North Loop,07:41:00,2021-01-04,4.0
1,AUTOTH,AUTOMOBILE THEFT,-93.279330,44.967045,Loring Park,09:00:00,2021-01-04,4.0
2,AUTOTH,AUTOMOBILE THEFT,-93.305827,44.982108,Harrison,10:00:00,2021-01-04,4.0
3,AUTOTH,AUTOMOBILE THEFT,-93.258186,44.967237,Elliot Park,13:19:00,2021-01-04,4.0
4,AUTOTH,AUTOMOBILE THEFT,-93.236780,44.981536,Marcy Holmes,13:30:00,2021-01-04,4.0
5,AUTOTH,AUTOMOBILE THEFT,-93.232335,44.951973,Longfellow,14:01:00,2021-01-04,4.0
6,AUTOTH,AUTOMOBILE THEFT,-93.307598,44.991504,Willard - Hay,14:21:00,2021-01-04,4.0
7,AUTOTH,AUTOMOBILE THEFT,-93.247317,44.924227,Ericsson,16:10:00,2021-01-04,4.0
8,AUTOTH,AUTOMOBILE THEFT,-93.313343,44.994257,Willard - Hay,16:00:00,2021-01-04,4.0
9,AUTOTH,AUTOMOBILE THEFT,-93.288902,44.930437,East Harriet,18:43:00,2021-01-04,4.0


In [30]:
#### add day of the week to dataframe
# df['dayOfWeek']=df['date'].dt.dayofweek()

# df['dayOfWeek'] = [i.dt.dayofweek() for i in df['date']]

# print(type(df['date'][0]))


calendar.day_name[df['date'][109].weekday()]


# df['date'][109].weekday()

df['dayOfWeek']=[calendar.day_name[df['date'][day].weekday()] for day in range(len(df['date']))]



df.head(50)

,offense,description,centerLong,centerLat,neighborhood,time,date,severity,dayOfWeek
0,AUTOTH,AUTOMOBILE THEFT,-93.280193,44.985047,North Loop,07:41:00,2021-01-04,4.0,Monday
1,AUTOTH,AUTOMOBILE THEFT,-93.279330,44.967045,Loring Park,09:00:00,2021-01-04,4.0,Monday
2,AUTOTH,AUTOMOBILE THEFT,-93.305827,44.982108,Harrison,10:00:00,2021-01-04,4.0,Monday
3,AUTOTH,AUTOMOBILE THEFT,-93.258186,44.967237,Elliot Park,13:19:00,2021-01-04,4.0,Monday
4,AUTOTH,AUTOMOBILE THEFT,-93.236780,44.981536,Marcy Holmes,13:30:00,2021-01-04,4.0,Monday
5,AUTOTH,AUTOMOBILE THEFT,-93.232335,44.951973,Longfellow,14:01:00,2021-01-04,4.0,Monday
6,AUTOTH,AUTOMOBILE THEFT,-93.307598,44.991504,Willard - Hay,14:21:00,2021-01-04,4.0,Monday
7,AUTOTH,AUTOMOBILE THEFT,-93.247317,44.924227,Ericsson,16:10:00,2021-01-04,4.0,Monday
8,AUTOTH,AUTOMOBILE THEFT,-93.313343,44.994257,Willard - Hay,16:00:00,2021-01-04,4.0,Monday
9,AUTOTH,AUTOMOBILE THEFT,-93.288902,44.930437,East Harriet,18:43:00,2021-01-04,4.0,Monday


In [ ]:
df.groupby('severity').count()

In [31]:
df.to_csv('2021_Clean.csv',index=False)